# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parent.parent.parent)
os.getcwd()

In [ ]:
import sys
sys.path.append('/home/tamir/workspace/automotive/src/python')

In [ ]:
import pandas as pd
import numpy as np
from typing import List
from pathlib import Path
import matplotlib.pyplot as plt

# Lanes Statistics

In [ ]:
import json

In [ ]:
def _has_road_boundaries(elem):
    if 'road_boundary_right' in elem and 'road_boundary_left' in elem:
        return True
    return False

def plot_set_statistics(eval_output_path, set_name):
    
    # load lanes json
    lanes_json_path = eval_output_path.joinpath('inputs/lanes.json')
    lanes = json.load(open(lanes_json_path))

    # load gt dataframe
    df = pd.read_csv(eval_output_path.joinpath('inputs/b2b_det_complete.tsv'), sep='\t')
    df['id'] = df.name.apply(lambda x: x.split('.')[0])

    total_frames = len(df.name.unique())
    total_lanes_found_per_frame = [len(lanes['frames'][key].keys()) if key in lanes['frames'] else 0 for key in df.id.unique()]
    has_both_boundaries = [_has_road_boundaries(lanes['frames'][elem]) for elem in lanes['frames'].keys()]
    
    bar_values = np.bincount(has_both_boundaries)
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(set_name)

    bars = axs[0].bar(range(len(bar_values)), bar_values, align='center')
    # Adding labels and title
    axs[0].set_xlabel('Has both boundaries')
    axs[0].set_ylabel('Counts')
    axs[0].set_title('Total has both boundaries')

    # Adding values on top of the bars
    for bar, count in zip(bars, bar_values):
        axs[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count),
                ha='center', va='bottom')

    # Displaying the plot

    bar_values = np.bincount(total_lanes_found_per_frame)
    bars = axs[1].bar(range(len(bar_values)), bar_values, align='center')

    # Adding labels and title
    axs[1].set_xlabel('Total found')
    axs[1].set_ylabel('Counts')
    axs[1].set_title('Total lanes found per frame')

    # Adding values on top of the bars
    for bar, count in zip(bars, bar_values):
        axs[1].text(bar.get_x() + bar.get_width() / 2, bar.get_height(), str(count),
                ha='center', va='bottom')

    # Displaying the plot
    plt.show()

In [ ]:
set_names = [
    '64f45788ac243656c2bd4986',
    '64b3729f628d0605eb5a082d',
    '64b63235628d0605eb5a1722',
    '64f561c6ac243656c2bd4fcd',
    '64f56176ac243656c2bd4fbb',
    '64f56183ac243656c2bd4fbe',
    
]

for set_name in set_names:
    eval_output_path = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval')
    plot_set_statistics(eval_output_path, set_name)

# Visualize Evaluations

For each set, visualizing the affect of lane filtering on precision & recall for different classes.
Main logic:
* Put any GT detection that is out of boundary as lanes ignore.
* Any prediction (prod) detection has grace margin to tackle cases where the detection is marked as ignore and gt is note (cause unwanted behaviour by the evaluation script)
* Peds has different logic than veichles, they has higher margin from boundary so only very far pedestrains are ignored. This is important duo to the fact that this filter filter almost all pedestrians.
* When no lanes are detected or when not both boundaries are present, the entire image is filtered.
* * When there is no cametra data it considered as no lanes.
* * When a lane has too few points inside an image (for example less than 20 out of 50) it is considered as None. Happens when a line is very horizontal instead of vertical.
* For GT detections, any object that is not in EGO \ NEXT RIGHT \ NEXT LEFT lane is ignored. Works only if the relevent lanes are present.

### Utils

In [ ]:
def calc_recall(x):
    a = x[1:].split('/')
    if int(a[1]) == 0:
        return 0
    return int(a[0]) / int(a[1])

In [ ]:
def plot_comparison(dfs: List[pd.DataFrame], class_name: str, title='Compare'):
    dfs = [df[df['class_name'] == class_name] for df in dfs]
    
    # Set up the subplot
    fig, axs = plt.subplots(1, 3, figsize=(20, 1 + 3 * len(dfs)))
    fig.suptitle(title)

    # Set the width of the bars
    bar_width = 0.2
    total_bars_width = (bar_width + 0.1) * len(dfs) + 0.2
    initial_positions = np.arange(len(dfs[0])) * total_bars_width
    
    columns = ['precision_loose', 'evaluated_recall', 'samples']
    
    # Set positions for the bars
    for i, df in enumerate(dfs):
        positions = initial_positions + i * (bar_width + 0.1)
        for k, col in enumerate(columns):
            axs[k].barh(positions, df[col], height=bar_width, label=df.name.values[0])
            for j, value in enumerate(df[col]):
                axs[k].text(value, positions[j], f'{value:.2f}', ha='right', va='center')
        
        # axs[0].barh(positions, df['precision_loose'], height=bar_width, label=df.name.values[0])
        # axs[1].barh(positions, df['evaluated_recall'], height=bar_width, label=df.name.values[0])
        # axs[2].barh(positions, df['samples'], height=bar_width, label=df.name.values[0])

        
            
        # for j, value in enumerate(df['evaluated_recall']):
        #     axs[1].text(value, positions[j], f'{value:.2f}', ha='right', va='center')

    # Set y-axis ticks and labels
    axs[0].set_yticks(initial_positions, dfs[0]['bin'])
    axs[0].set_xlabel('Precision')
    axs[0].set_ylabel('Bins')
    axs[0].set_title(f'Precision: {class_name}')
    axs[0].legend()  # Display legend
    
    # Set y-axis ticks and labels
    axs[1].set_yticks(initial_positions, dfs[0]['bin'])
    axs[1].set_xlabel('Recall')
    axs[1].set_ylabel('Bins')
    axs[1].set_title(f'Recall: {class_name}')
    axs[1].legend()  # Display legend

    # Set y-axis ticks and labels
    axs[2].set_yticks(initial_positions, dfs[0]['bin'])
    axs[2].set_xlabel('Samples')
    axs[2].set_ylabel('Bins')
    axs[2].set_title(f'Samples: {class_name}')
    axs[2].legend()  # Display legend

    # Adjust layout for better spacing
    plt.tight_layout()

    # Display the plot
    plt.show()
    

## Baseline

In [ ]:
baselines = {}

### 64f56183ac243656c2bd4fbe

In [ ]:
eval_baseline = Path('/home/tamir/workspace/AB_B2B_Eval/tmp/64f56183ac243656c2bd4fbe_eval_baseline')
df_baseline = pd.read_csv(eval_baseline.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_baseline['name'] = 'baseline'
df_baseline['evaluated_recall'] = df_baseline.recall.apply(calc_recall)
baselines['64f56183ac243656c2bd4fbe'] = df_baseline

plot_comparison(dfs=[baselines['64f56183ac243656c2bd4fbe']], class_name='4w', title='64f56183ac243656c2bd4fbe')

### 64f561c6ac243656c2bd4fcd

In [ ]:

eval_baseline = Path('/home/tamir/workspace/AB_B2B_Eval/tmp/64f561c6ac243656c2bd4fcd_eval_baseline')
df_baseline = pd.read_csv(eval_baseline.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_baseline['name'] = 'baseline'
df_baseline['evaluated_recall'] = df_baseline.recall.apply(calc_recall)
baselines['64f561c6ac243656c2bd4fcd'] = df_baseline

plot_comparison(dfs=[baselines['64f561c6ac243656c2bd4fcd']], class_name='4w', title='64f561c6ac243656c2bd4fcd')

### 64f45788ac243656c2bd4986

In [ ]:
set_name = '64f45788ac243656c2bd4986'
eval_baseline = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_baseline')
df_baseline = pd.read_csv(eval_baseline.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_baseline['name'] = 'baseline'
df_baseline['evaluated_recall'] = df_baseline.recall.apply(calc_recall)
baselines[set_name] = df_baseline

plot_comparison(dfs=[baselines[set_name]], class_name='4w', title=set_name)

## Eval with Lanes (v1)

In [ ]:
results_with_lanes_v1 = {}

### 64f56183ac243656c2bd4fbe

In [ ]:
set_name = '64f56183ac243656c2bd4fbe'

eval_with_lanes_v1 = Path(f'/home/tamir/s3_sync/europe_run_3_10/{set_name}/eval_on_infer_0.1.4/prod_sf_vs_al')
df_with_lanes_v1 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v1['name'] = 'with_lanes'
df_with_lanes_v1['evaluated_recall'] = df_with_lanes_v1.recall.apply(calc_recall)
results_with_lanes_v1[set_name] = df_with_lanes_v1

dfs = [baselines[set_name], results_with_lanes_v1[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

### 64f561c6ac243656c2bd4fcd

In [ ]:
set_name = '64f561c6ac243656c2bd4fcd'

eval_with_lanes_v1 = Path(f'/home/tamir/s3_sync/europe_run_3_10/{set_name}/eval_on_infer_0.1.4/prod_sf_vs_al')
df_with_lanes_v1 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v1['name'] = 'with_lanes'
df_with_lanes_v1['evaluated_recall'] = df_with_lanes_v1.recall.apply(calc_recall)
results_with_lanes_v1[set_name] = df_with_lanes_v1

dfs = [baselines[set_name], results_with_lanes_v1[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

### 64f45788ac243656c2bd4986

In [ ]:
set_name = '64f45788ac243656c2bd4986'

eval_with_lanes_v1 = Path(f'/home/tamir/s3_sync/europe_run_3_10/{set_name}/eval_on_infer_0.1.4/prod_sf_vs_al')
df_with_lanes_v1 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v1['name'] = 'with_lanes'
df_with_lanes_v1['evaluated_recall'] = df_with_lanes_v1.recall.apply(calc_recall)
results_with_lanes_v1[set_name] = df_with_lanes_v1

dfs = [baselines[set_name], results_with_lanes_v1[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

## Eval with lanes (v2)

In [ ]:
results_with_lanes_v2 = {}

### 64f56183ac243656c2bd4fbe

In [ ]:
set_name = '64f56183ac243656c2bd4fbe'

eval_with_lanes_v2 = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v2')
df_with_lanes_v2 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v2['name'] = 'with_lanes'
df_with_lanes_v2['evaluated_recall'] = df_with_lanes_v2.recall.apply(calc_recall)
results_with_lanes_v2[set_name] = df_with_lanes_v2

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

### 64f561c6ac243656c2bd4fcd

In [ ]:
set_name = '64f561c6ac243656c2bd4fcd'

eval_with_lanes_v2 = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v2')
df_with_lanes_v2 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v2['name'] = 'with_lanes_v2'
df_with_lanes_v2['evaluated_recall'] = df_with_lanes_v2.recall.apply(calc_recall)
results_with_lanes_v2[set_name] = df_with_lanes_v2

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

### 64f45788ac243656c2bd4986

In [ ]:
set_name = '64f45788ac243656c2bd4986'

eval_with_lanes_v2 = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v2')
df_with_lanes_v2 = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df_with_lanes_v2['name'] = 'with_lanes_v2'
df_with_lanes_v2['evaluated_recall'] = df_with_lanes_v2.recall.apply(calc_recall)
results_with_lanes_v2[set_name] = df_with_lanes_v2

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

Following the next pivot example, there are cases where there is a detection by prod that is out of boundaries.
This detection does not have an ignore GT

## Eval with lanes (v3)

### Reducing grace gap

In [ ]:
results_with_lanes_v3 = {}

### 64f561c6ac243656c2bd4fcd

In [ ]:
set_name = '64f561c6ac243656c2bd4fcd'

eval = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v3')
df = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df['name'] = 'with_lanes_v3'
df['evaluated_recall'] = df.recall.apply(calc_recall)
results_with_lanes_v3[set_name] = df

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name], results_with_lanes_v3[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

### 64f56183ac243656c2bd4fbe

In [ ]:
set_name = '64f56183ac243656c2bd4fbe'

eval = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v3')
df = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df['name'] = 'with_lanes_v3'
df['evaluated_recall'] = df.recall.apply(calc_recall)
results_with_lanes_v3[set_name] = df

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name], results_with_lanes_v3[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

#### 64f45788ac243656c2bd4986

In [ ]:
set_name = '64f45788ac243656c2bd4986'

eval = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/{set_name}_eval_v3')
df = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df['name'] = 'with_lanes_v3'
df['evaluated_recall'] = df.recall.apply(calc_recall)
results_with_lanes_v3[set_name] = df

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name], results_with_lanes_v3[set_name]]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)

In [ ]:
set_name = '64f45788ac243656c2bd4986'

eval = Path(f'/home/tamir/workspace/AB_B2B_Eval/tmp/64f45788ac243656c2bd4986_eval_test')
df = pd.read_csv(eval_with_lanes_v1.joinpath('summary/eval_table_results_max_recall_test.tsv'), sep='\t')
df['name'] = 'with_lanes_test'
df['evaluated_recall'] = df.recall.apply(calc_recall)
results_with_lanes_v3['test'] = df

dfs = [baselines[set_name], results_with_lanes_v1[set_name], results_with_lanes_v2[set_name], results_with_lanes_v3['test']]
plot_comparison(dfs=dfs, class_name='4w', title=set_name)
plot_comparison(dfs=dfs, class_name='peds', title=set_name)